In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier

import matplotlib
import matplotlib.pyplot as plt
from pylab import imshow

import h5py
import sys
import os, os.path
import pickle

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tslearn.metrics import dtw as dtw_metric
# https://dtaidistance.readthedocs.io/en/latest/usage/dtw.html
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis


from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow as tf
from keras.layers import Conv2D
from keras.layers import MaxPooling2D


sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
import NASA_plot_core as rcp

In [2]:
def DTW_prune(ts1, ts2):
    d,_ = dtw.warping_paths(ts1, ts2, window=10, use_pruning=True);
    return d

### Read Meta

In [3]:
meta_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
meta = pd.read_csv(meta_dir+"evaluation_set.csv")
meta_moreThan10Acr=meta[meta.ExctAcr>10]
print (meta.shape)
print (meta_moreThan10Acr.shape)
meta.head(2)

(6340, 8)
(3539, 8)


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [4]:
non_expert_vote_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/badData/"

out_name = non_expert_vote_dir + "nonExpert_votes.csv"
nonExpert_V= pd.read_csv(out_name)

out_name = non_expert_vote_dir + "limitCrops_nonExpert_votes.csv"
limitCrops_nonExpert_V = pd.read_csv(out_name)

In [5]:
limitCrops_nonExpert_V.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Vote,NDVI_TS_Name,CropTyp,Irrigtn,DataSrc,Acres,LstSrvD,county
0,31,3,53189_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,none,none,single,AdamBenton2016_53189_WSDA_SF_2016.png,barley,center pivot,wsda,132,2016/09/20 00:00:00,Adams
1,31,4,101534_WSDA_SF_2018,Single Crop,Double Crop,Single Crop,none,none,single,FranklinYakima2018_101534_WSDA_SF_2018.png,barley,center pivot,wsda,3,2018/08/14 00:00:00,Franklin


In [6]:
bad_columns = ['NDVI_TS_Name', 'Acres', 'LstSrvD', 'Form', 'Question']

nonExpert_V.drop(bad_columns, axis='columns', inplace=True)
limitCrops_nonExpert_V.drop(bad_columns, axis='columns', inplace=True)

In [7]:
limitCrops_nonExpert_V.head(2)

,ID,Hossein,Supriya,Kirti,Mike,Min,Vote,CropTyp,Irrigtn,DataSrc,county
0,53189_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,none,none,single,barley,center pivot,wsda,Adams
1,101534_WSDA_SF_2018,Single Crop,Double Crop,Single Crop,none,none,single,barley,center pivot,wsda,Franklin


In [8]:
nonExpert_V=pd.merge(nonExpert_V, 
                     meta[["ID", "ExctAcr"]],
                     on=['ID'], how='left')


limitCrops_nonExpert_V=pd.merge(limitCrops_nonExpert_V, 
                                meta[["ID", "ExctAcr"]],
                                on=['ID'], how='left')

In [9]:
print (len(nonExpert_V.ID.unique()))
print (len(nonExpert_V[nonExpert_V.ExctAcr<10].ID.unique()))

5036
2401


# Toss small fields

In [10]:
nonExpert_V=nonExpert_V[nonExpert_V.ExctAcr>10]
limitCrops_nonExpert_V=limitCrops_nonExpert_V[limitCrops_nonExpert_V.ExctAcr>10]

### Drop unknown field

In [16]:
nonExpert_V=nonExpert_V[nonExpert_V.CropTyp!="unknown"].copy()
limitCrops_nonExpert_V=limitCrops_nonExpert_V[limitCrops_nonExpert_V.CropTyp!="unknown"].copy()

# Write the non-expert set to the disk

In [58]:
out_name = meta_dir + "nonExpert_2605_votes.csv"
nonExpert_V.to_csv(out_name, index = False)

# Read SG files

In [17]:
VI_idx = "EVI"
data_dir = "/Users/hn/Documents/01_research_data/NASA/VI_TS/05_SG_TS/"

In [18]:
file_names = ["SG_Walla2015_" + VI_idx + "_JFD.csv", "SG_AdamBenton2016_" + VI_idx + "_JFD.csv", 
              "SG_Grant2017_" + VI_idx + "_JFD.csv", "SG_FranklinYakima2018_"+ VI_idx +"_JFD.csv"]

data=pd.DataFrame()

for file in file_names:
    curr_file=pd.read_csv(data_dir + file)
    curr_file['human_system_start_time'] = pd.to_datetime(curr_file['human_system_start_time'])
    
    # These data are for 3 years. The middle one is the correct one
    all_years = sorted(curr_file.human_system_start_time.dt.year.unique())
    if len(all_years)==3 or len(all_years)==2:
        proper_year = all_years[1]
    elif len(all_years)==1:
        proper_year = all_years[0]

    curr_file = curr_file[curr_file.human_system_start_time.dt.year==proper_year]
    data=pd.concat([data, curr_file])

data.reset_index(drop=True, inplace=True)
data=data[data.ID.isin(list(nonExpert_V.ID.unique()))]
data.reset_index(drop=True, inplace=True)

data.head(2)

,ID,human_system_start_time,EVI
0,144551_WSDA_SF_2015,2015-01-10,0.051819
1,144551_WSDA_SF_2015,2015-01-20,0.054279


# Sort the order of time-series and labels identically

In [19]:
data.sort_values(by=["ID", 'human_system_start_time'], inplace=True)
nonExpert_V.sort_values(by=["ID"], inplace=True)

data.reset_index(drop=True, inplace=True)
nonExpert_V.reset_index(drop=True, inplace=True)

assert (len(data.ID.unique()) == len(nonExpert_V.ID.unique()))


# Widen

In [20]:
EVI_colnames = [VI_idx + "_" + str(ii) for ii in range(1, 37) ]
columnNames = ["ID"] + EVI_colnames
data_wide = pd.DataFrame(columns=columnNames, 
                                index=range(len(data.ID.unique())))
data_wide["ID"] = data.ID.unique()

for an_ID in data.ID.unique():
    curr_df = data[data.ID==an_ID]
    
    data_wide_indx = data_wide[data_wide.ID==an_ID].index
    data_wide.loc[data_wide_indx, "EVI_1":"EVI_36"] = curr_df.EVI.values[:36]

In [21]:
data_wide.head(2)

,ID,EVI_1,EVI_2,EVI_3,EVI_4,EVI_5,EVI_6,EVI_7,EVI_8,EVI_9,...,EVI_27,EVI_28,EVI_29,EVI_30,EVI_31,EVI_32,EVI_33,EVI_34,EVI_35,EVI_36
0,100041_WSDA_SF_2017,0.190623,0.136657,0.128576,0.187575,0.194632,0.211836,0.251309,0.293089,0.333297,...,0.327493,0.342592,0.333237,0.271697,0.173352,0.108954,0.096683,0.141439,0.187477,0.199279
1,100048_WSDA_SF_2017,0.0,0.0,0.0,0.0,-0.006879,0.00189,0.022394,0.058236,0.084847,...,0.117901,0.118943,0.10218,0.083567,0.060726,0.043216,0.044677,0.043529,0.044976,0.047556


In [22]:
nonExpert_V.head(2)

,ID,Hossein,Supriya,Kirti,Mike,Min,Vote,CropTyp,Irrigtn,DataSrc,county,ExctAcr
0,100041_WSDA_SF_2017,Single Crop,Single Crop,Single Crop,Single Crop,Unsure,single,"grape, wine",drip,wsda,Grant,23.441439
1,100048_WSDA_SF_2017,Single Crop,Single Crop,Single Crop,Unsure,none,single,"bean, green",rill,wsda,Grant,18.033240


In [23]:
nonExpert_V_clean=nonExpert_V[["ID", "Vote", "CropTyp", "Irrigtn", "county", "ExctAcr"]].copy()
nonExpert_V_clean.replace("double", 2, inplace=True)
nonExpert_V_clean.replace("single", 1, inplace=True)

In [24]:
print (sorted(nonExpert_V_clean.CropTyp.unique()))

['alfalfa hay', 'apple', 'apricot', 'asparagus', 'barley', 'barley hay', 'bean, dry', 'bean, green', 'blueberry', 'bluegrass seed', 'buckwheat', 'canola', 'carrot', 'carrot seed', 'cherry', 'corn seed', 'corn, field', 'corn, sweet', 'fallow', 'fallow, idle', 'fallow, tilled', 'fescue seed', 'grape, juice', 'grape, wine', 'grass hay', 'grass seed', 'hops', 'market crops', 'medicinal herb', 'mint', 'mustard', 'nectarine/peach', 'oat hay', 'onion', 'onion seed', 'orchard, unknown', 'pasture', 'pea seed', 'pea, dry', 'pea, green', 'pear', 'pepper', 'plum', 'poplar', 'potato', 'pumpkin', 'ryegrass seed', 'sod farm', 'squash', 'sudangrass', 'sugar beet', 'sugar beet seed', 'sunflower', 'sunflower seed', 'timothy', 'triticale', 'triticale hay', 'watermelon', 'wheat', 'wheat fallow', 'wildlife feed', 'yellow mustard']


In [25]:
len(nonExpert_V_clean.ID.unique())

2605

# SVM

In [26]:
model_dir = "/Users/hn/Documents/01_research_data/NASA/ML_Models/"
filename = model_dir + 'SVM_classifier_balanced_SGEVI_00.sav'
SVM_classifier_balanced_00 = pickle.load(open(filename, 'rb'))

In [27]:
SVM_classifier_balanced_00_predictions = SVM_classifier_balanced_00.predict(data_wide.iloc[:, 1:])

In [37]:
#### add SVM result to vote table
nonExpert_V_clean["SVM_balanced_pred"]=SVM_classifier_balanced_00_predictions

In [38]:
nonExpert_V_clean.head(2)

,ID,Vote,CropTyp,Irrigtn,county,ExctAcr,SVM_balanced_pred
0,100041_WSDA_SF_2017,1,"grape, wine",drip,Grant,23.441439,1
1,100048_WSDA_SF_2017,1,"bean, green",rill,Grant,18.033240,1


# kNN
Winner is **uniform** weight.

In [30]:
def DTW_prune(ts1, ts2):
    d,_ = dtw.warping_paths(ts1, ts2, window=10, use_pruning=True);
    return d

In [31]:
filename = model_dir + "00_KNN_SG_EVI_DTW_prune_uniformWeight_9NNisBest.sav"
uniform_KNN = pickle.load(open(filename, 'rb'))

In [32]:
%%time
KNN_DTW_test_predictions_uniform = uniform_KNN.predict(data_wide.iloc[:, 1:])

CPU times: user 55min 17s, sys: 14.8 s, total: 55min 32s
Wall time: 56min 4s


In [51]:
nonExpert_V_clean["knn_uniform_pred"]=KNN_DTW_test_predictions_uniform
nonExpert_V_clean.head(2)

,ID,Vote,CropTyp,Irrigtn,county,ExctAcr,SVM_balanced_pred,knn_uniform_pred
0,100041_WSDA_SF_2017,1,"grape, wine",drip,Grant,23.441439,1,1
1,100048_WSDA_SF_2017,1,"bean, green",rill,Grant,18.033240,1,1


In [53]:
sum(nonExpert_V_clean.SVM_balanced_pred==nonExpert_V_clean.knn_uniform_pred)

2522

# RF

In [43]:
%%time
filename = model_dir + 'SG_forest_grid_1.sav'
forest_grid_1_SG = pickle.load(open(filename, 'rb'))
RF_grid_1_predictions = forest_grid_1_SG.predict(data_wide.iloc[:, 1:])

CPU times: user 52.9 ms, sys: 30.4 ms, total: 83.3 ms
Wall time: 102 ms


In [55]:
nonExpert_V_clean["RF_G1_pred"]=RF_grid_1_predictions
nonExpert_V_clean.head(2)

,ID,Vote,CropTyp,Irrigtn,county,ExctAcr,SVM_balanced_pred,knn_uniform_pred,RF_G1_pred
0,100041_WSDA_SF_2017,1,"grape, wine",drip,Grant,23.441439,1,1,1
1,100048_WSDA_SF_2017,1,"bean, green",rill,Grant,18.033240,1,1,1


In [56]:
sum(nonExpert_V_clean.SVM_balanced_pred==nonExpert_V_clean.RF_G1_pred)

2544

# DL

We are going with **prob_point7**.

In [87]:
from keras.models import load_model
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array

def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(224, 224))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 224, 224, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img

In [80]:
model_dir = "/Users/hn/Documents/01_research_data/NASA/ML_Models/"
DL_model = load_model(model_dir+'01_TL_SingleDoubleEVI_SG_train80.h5')

In [ ]:
### We did this once. We can read from now on. 

# SG_nonExpert_V_EVI_plot_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/SG_nonExpert_V_EVI/"

# nonExpert_df_filenames = os.listdir(SG_nonExpert_V_EVI_plot_dir)
# nonExpert_df = pd.DataFrame({'filename': nonExpert_df_filenames})
# nb_samples = nonExpert_df.shape[0]

# nonExpert_df["human_predict"] = nonExpert_df.filename.str.split("_", expand=True)[0]
# nonExpert_df["prob_single"]=-1.0
# print (nonExpert_df.shape)
# nonExpert_df.head(2)


# for idx in nonExpert_df.index:
#     img = load_image(SG_nonExpert_V_EVI_plot_dir + nonExpert_df.loc[idx, 'filename']);
#     nonExpert_df.loc[idx, 'prob_single'] = DL_model.predict(img, verbose=False)[0][0];

# for prob in [0.3, 0.4, 0.5, 0.6, 0.7]:
#     colName = "prob_point"+str(prob)[2:]
#     nonExpert_df.loc[nonExpert_df.prob_single<prob, colName] = 'double'
#     nonExpert_df.loc[nonExpert_df.prob_single>=prob, colName] = 'single'

# nonExpert_df.rename(columns={"filename": "ID"}, inplace=True)
# nonExpert_df.rename(columns={"human_predict": "Vote"}, inplace=True) 
# ID = [x[0] for x in nonExpert_df.ID.str.split(".")]
# nonExpert_df.ID=ID

# ID = ["_".join(x[1:]) for x in nonExpert_df.ID.str.split("_")]
# nonExpert_df.ID=ID

# nonExpert_df.replace("double", 2, inplace=True)
# nonExpert_df.replace("single", 1, inplace=True)

# nonExpert_df.head(2)

# out_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/SG_nonExpert_2605_V_EVI_result/"
# os.makedirs(out_dir, exist_ok=True)
# out_name = out_dir + "01_TL_nonExpert_2605_predictions_" + VI_idx + "_SG.csv"
# nonExpert_df.to_csv(out_name, index = False)

In [111]:
out_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/SG_nonExpert_2605_V_EVI_result/"
out_name = out_dir + "01_TL_nonExpert_2605_predictions_" + VI_idx + "_SG.csv"
nonExpert_df=pd.read_csv(out_name)
nonExpert_df.head(2)

,ID,Vote,prob_single,prob_point3,prob_point4,prob_point5,prob_point6,prob_point7
0,108316_WSDA_SF_2017,2,0.999967,1,1,1,1,1
1,59032_WSDA_SF_2016,2,1.000000,1,1,1,1,1


In [113]:
nonExpert_V_clean=pd.merge(nonExpert_V_clean, nonExpert_df[["ID", "prob_point7"]],
                           on=['ID'], how='left')

In [116]:
nonExpert_V_clean.rename(columns={"prob_point7": "DL_pred"}, inplace=True)
nonExpert_V_clean.head(2)

,ID,Vote,CropTyp,Irrigtn,county,ExctAcr,SVM_balanced_pred,knn_uniform_pred,RF_G1_pred,DL_pred
0,100041_WSDA_SF_2017,1,"grape, wine",drip,Grant,23.441439,1,1,1,1
1,100048_WSDA_SF_2017,1,"bean, green",rill,Grant,18.033240,1,1,1,1


In [117]:
nonExpert_V_clean.shape

(2605, 10)

In [142]:
SVM_A1P1=nonExpert_V_clean.copy()
SVM_A1P1=SVM_A1P1[SVM_A1P1.Vote==1]
SVM_A1P1=SVM_A1P1[SVM_A1P1.SVM_balanced_pred==1]
print ("There are [{0}] fields under SVM A1P1.".format(SVM_A1P1.shape[0]))
print ("Area is [{0:.2f}] under SVM A1P1.".format(SVM_A1P1.ExctAcr.sum()))

print ("=======================================================")
SVM_A2P2=nonExpert_V_clean.copy()
SVM_A2P2=SVM_A2P2[SVM_A2P2.Vote==2]
SVM_A2P2=SVM_A2P2[SVM_A2P2.SVM_balanced_pred==2]
print ("There are [{0}] fields under SVM A1P1.".format(SVM_A2P2.shape[0]))
print ("Area is [{0:.2f}] under SVM A1P1.".format(SVM_A2P2.ExctAcr.sum()))
print ("=======================================================")


SVM_A2P1=nonExpert_V_clean.copy()
SVM_A2P1=SVM_A2P1[SVM_A2P1.Vote==2]
SVM_A2P1=SVM_A2P1[SVM_A2P1.SVM_balanced_pred==1]
print ("There are [{0}] fields under SVM A2P1.".format(SVM_A2P1.shape[0]))
print ("Area is [{0:.2f}] under SVM A2P1.".format(SVM_A2P1.ExctAcr.sum()))

print ("=======================================================")
SVM_A1P2=nonExpert_V_clean.copy()
SVM_A1P2=SVM_A1P2[SVM_A1P2.Vote==1]
SVM_A1P2=SVM_A1P2[SVM_A1P2.SVM_balanced_pred==2]
print ("There are [{0}] fields under SVM A1P2.".format(SVM_A1P2.shape[0]))
print ("Area is [{0:.2f}] under SVM A1P2.".format(SVM_A1P2.ExctAcr.sum()))
print ("=======================================================")

There are [2308] fields under SVM A1P1.
Area is [92598.49] under SVM A1P1.
There are [172] fields under SVM A1P1.
Area is [11712.18] under SVM A1P1.
There are [65] fields under SVM A2P1.
Area is [4575.99] under SVM A2P1.
There are [60] fields under SVM A1P2.
Area is [2489.52] under SVM A1P2.


In [143]:
RF_A1P1=nonExpert_V_clean.copy()
RF_A1P1=RF_A1P1[RF_A1P1.Vote==1]
RF_A1P1=RF_A1P1[RF_A1P1.RF_G1_pred==1]
print ("There are [{0}] fields under RF A1P1.".format(RF_A1P1.shape[0]))
print ("Area is [{0:.2f}] under RF A1P1.".format(RF_A1P1.ExctAcr.sum()))

print ("=======================================================")
RF_A2P2=nonExpert_V_clean.copy()
RF_A2P2=RF_A2P2[RF_A2P2.Vote==2]
RF_A2P2=RF_A2P2[RF_A2P2.RF_G1_pred==2]
print ("There are [{0}] fields under RF A1P1.".format(RF_A2P2.shape[0]))
print ("Area is [{0:.2f}] under RF A1P1.".format(RF_A2P2.ExctAcr.sum()))
print ("=======================================================")


RF_A2P1=nonExpert_V_clean.copy()
RF_A2P1=RF_A2P1[RF_A2P1.Vote==2]
RF_A2P1=RF_A2P1[RF_A2P1.RF_G1_pred==1]
print ("There are [{0}] fields under RF A2P1.".format(RF_A2P1.shape[0]))
print ("Area is [{0:.2f}] under RF A2P1.".format(RF_A2P1.ExctAcr.sum()))

print ("=======================================================")
RF_A1P2=nonExpert_V_clean.copy()
RF_A1P2=RF_A1P2[RF_A1P2.Vote==1]
RF_A1P2=RF_A1P2[RF_A1P2.RF_G1_pred==2]
print ("There are [{0}] fields under RF A1P2.".format(RF_A1P2.shape[0]))
print ("Area is [{0:.2f}] under RF A1P2.".format(RF_A1P2.ExctAcr.sum()))
print ("=======================================================")

There are [2320] fields under RF A1P1.
Area is [93446.38] under RF A1P1.
There are [165] fields under RF A1P1.
Area is [11091.41] under RF A1P1.
There are [72] fields under RF A2P1.
Area is [5196.76] under RF A2P1.
There are [48] fields under RF A1P2.
Area is [1641.63] under RF A1P2.


In [145]:
knn_A1P1=nonExpert_V_clean.copy()
knn_A1P1=knn_A1P1[knn_A1P1.Vote==1]
knn_A1P1=knn_A1P1[knn_A1P1.knn_uniform_pred==1]
print ("There are [{0}] fields under knn A1P1.".format(knn_A1P1.shape[0]))
print ("Area is [{0:.2f}] under knn A1P1.".format(knn_A1P1.ExctAcr.sum()))

print ("=======================================================")
knn_A2P2=nonExpert_V_clean.copy()
knn_A2P2=knn_A2P2[knn_A2P2.Vote==2]
knn_A2P2=knn_A2P2[knn_A2P2.knn_uniform_pred==2]
print ("There are [{0}] fields under knn A1P1.".format(knn_A2P2.shape[0]))
print ("Area is [{0:.2f}] under knn A1P1.".format(knn_A2P2.ExctAcr.sum()))
print ("=======================================================")


knn_A2P1=nonExpert_V_clean.copy()
knn_A2P1=knn_A2P1[knn_A2P1.Vote==2]
knn_A2P1=knn_A2P1[knn_A2P1.knn_uniform_pred==1]
print ("There are [{0}] fields under knn A2P1.".format(knn_A2P1.shape[0]))
print ("Area is [{0:.2f}] under knn A2P1.".format(knn_A2P1.ExctAcr.sum()))

print ("=======================================================")
knn_A1P2=nonExpert_V_clean.copy()
knn_A1P2=knn_A1P2[knn_A1P2.Vote==1]
knn_A1P2=knn_A1P2[knn_A1P2.knn_uniform_pred==2]
print ("There are [{0}] fields under knn A1P2.".format(knn_A1P2.shape[0]))
print ("Area is [{0:.2f}] under knn A1P2.".format(knn_A1P2.ExctAcr.sum()))
print ("=======================================================")

There are [2350] fields under knn A1P1.
Area is [93952.77] under knn A1P1.
There are [169] fields under knn A1P1.
Area is [11516.36] under knn A1P1.
There are [68] fields under knn A2P1.
Area is [4771.80] under knn A2P1.
There are [18] fields under knn A1P2.
Area is [1135.24] under knn A1P2.


In [147]:
DL_A1P1=nonExpert_V_clean.copy()
DL_A1P1=DL_A1P1[DL_A1P1.Vote==1]
DL_A1P1=DL_A1P1[DL_A1P1.DL_pred==1]
print ("There are [{0}] fields under DL A1P1.".format(DL_A1P1.shape[0]))
print ("Area is [{0:.2f}] under DL A1P1.".format(DL_A1P1.ExctAcr.sum()))

print ("=======================================================")
DL_A2P2=nonExpert_V_clean.copy()
DL_A2P2=DL_A2P2[DL_A2P2.Vote==2]
DL_A2P2=DL_A2P2[DL_A2P2.DL_pred==2]
print ("There are [{0}] fields under DL A1P1.".format(DL_A2P2.shape[0]))
print ("Area is [{0:.2f}] under DL A1P1.".format(DL_A2P2.ExctAcr.sum()))
print ("=======================================================")


DL_A2P1=nonExpert_V_clean.copy()
DL_A2P1=DL_A2P1[DL_A2P1.Vote==2]
DL_A2P1=DL_A2P1[DL_A2P1.DL_pred==1]
print ("There are [{0}] fields under DL A2P1.".format(DL_A2P1.shape[0]))
print ("Area is [{0:.2f}] under DL A2P1.".format(DL_A2P1.ExctAcr.sum()))

print ("=======================================================")
DL_A1P2=nonExpert_V_clean.copy()
DL_A1P2=DL_A1P2[DL_A1P2.Vote==1]
DL_A1P2=DL_A1P2[DL_A1P2.DL_pred==2]
print ("There are [{0}] fields under DL A1P2.".format(DL_A1P2.shape[0]))
print ("Area is [{0:.2f}] under DL A1P2.".format(DL_A1P2.ExctAcr.sum()))
print ("=======================================================")

There are [2347] fields under DL A1P1.
Area is [93871.40] under DL A1P1.
There are [171] fields under DL A1P1.
Area is [11727.89] under DL A1P1.
There are [66] fields under DL A2P1.
Area is [4560.28] under DL A2P1.
There are [21] fields under DL A1P2.
Area is [1216.61] under DL A1P2.


In [149]:
nonExpert_V_clean.ExctAcr.sum()

111376.17647069224

In [155]:
(3344/111376)*100

3.0024421778480104